In [4]:
from pathlib import Path
import re
import joblib
import gmaps
import googlemaps
from dotenv import load_dotenv
import googlemaps
import ast

In [5]:
load_dotenv()
toronto_coord = (43.651070,-79.347015)
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])

origin = (43.660849, -79.295494) # 27 King's College Cir, Toronto, ON M5S
dst = (43.64764,-79.398266) # 484 Richmond St W, Toronto, ON M5V 1Y2
maps = googlemaps.Client(key=os.environ['GOOGLE_API_KEY'])
figs = gmaps.figure(center=toronto_coord,zoom_level=11)

In [6]:
def con_gps(addr):
    import googlemaps
    maps = googlemaps.Client(key=os.environ['GOOGLE_API_KEY'])
    res = maps.geocode(addr)[0]
    lat, long = res['geometry']['location'].values()
    return float(lat), float(long)

# Calculate Midpoint and Radius
def calc_midpt_radius(latitude1, longitude1, latitude2, longitude2):
    earth_radius = 3959*5280.0
    
    Lat1 = (latitude1)*(np.math.pi/180.0)
    Long1 = (longitude1)*(np.math.pi/180.0)
    Lat2 = (latitude2)*(np.math.pi/180.0)
    Long2 = (longitude2)*(np.math.pi/180.0)
    
    dist = 2.0*earth_radius*np.arcsin(np.sqrt(np.power(np.math.sin((Lat2 - Lat1)/2.0),2.0)+np.math.cos(Lat1)*np.math.cos(Lat2)*np.power(np.math.sin((Long2 - Long1)/2.0),2.0)))
    #dis = np.sqrt((Lat1-Lat2)**2 +(Long1-Long2)**2)
    radius = dist/2.0
    #radius = radius+100.0
    
    mid_lat = (latitude1+latitude2)/2.0
    mid_long = (longitude1+longitude2)/2.0
            
    return mid_lat, mid_long, radius
   # returns center coordinate and radius in ft (plus 100 ft)

In [109]:
lat1, long1 = con_gps("27 King's College Cir, Toronto, ON M5S")
lat2, long2 = con_gps("235 Yonge St Toronto, ON M5B 1N8")
midlat, midlong, r = calc_midpt_radius(lat1, long1, lat2, long2)

In [110]:
midlat, midlong, r

(43.65759675, -79.38761160000001, 2395.190112742883)

In [111]:
res = maps.reverse_geocode((midlat, midlong))
for item in res:
    with open('output_loc.txt', 'a') as f:
        f.write(f"{item['geometry']['location']['lat']},{item['geometry']['location']['lng']}\n")

In [112]:
s_loc_t = np.genfromtxt("./output_loc.txt", delimiter=',', dtype=float)
s_loc_t

array([[  43.654304 ,  -79.3969122],
       [  43.6543878,  -79.3969242],
       [  43.654296 ,  -79.396791 ],
       [  43.6541698,  -79.3967243],
       [  43.6541982,  -79.3970905],
       [  43.6543596,  -79.3973226],
       [  43.6537341,  -79.3930124],
       [  43.6532057,  -79.4000493],
       [  43.6486795,  -79.3803231],
       [  43.653226 ,  -79.3831843],
       [  43.6689775,  -79.2902133],
       [  51.253775 ,  -85.323214 ],
       [  56.130366 , -106.346771 ],
       [  43.654304 ,  -79.3969122],
       [  43.6543878,  -79.3969242],
       [  43.654296 ,  -79.396791 ],
       [  43.6541698,  -79.3967243],
       [  43.6541982,  -79.3970905],
       [  43.6543596,  -79.3973226],
       [  43.6537341,  -79.3930124],
       [  43.6532057,  -79.4000493],
       [  43.6486795,  -79.3803231],
       [  43.653226 ,  -79.3831843],
       [  43.6689775,  -79.2902133],
       [  51.253775 ,  -85.323214 ],
       [  56.130366 , -106.346771 ],
       [  43.654304 ,  -79.3969122],
 

In [113]:
s_loc = np.zeros((np.shape(s_loc_t)[0]+2, 2))
s_loc[0,0]=lat1
s_loc[0,1] = long1
s_loc[1:-1, :] = s_loc_t[:, 0:2]
s_loc[-1,0] = lat2
s_loc[-1,1] = long2
s_loc

array([[  43.660849 ,  -79.3954943],
       [  43.654304 ,  -79.3969122],
       [  43.6543878,  -79.3969242],
       [  43.654296 ,  -79.396791 ],
       [  43.6541698,  -79.3967243],
       [  43.6541982,  -79.3970905],
       [  43.6543596,  -79.3973226],
       [  43.6537341,  -79.3930124],
       [  43.6532057,  -79.4000493],
       [  43.6486795,  -79.3803231],
       [  43.653226 ,  -79.3831843],
       [  43.6689775,  -79.2902133],
       [  51.253775 ,  -85.323214 ],
       [  56.130366 , -106.346771 ],
       [  43.654304 ,  -79.3969122],
       [  43.6543878,  -79.3969242],
       [  43.654296 ,  -79.396791 ],
       [  43.6541698,  -79.3967243],
       [  43.6541982,  -79.3970905],
       [  43.6543596,  -79.3973226],
       [  43.6537341,  -79.3930124],
       [  43.6532057,  -79.4000493],
       [  43.6486795,  -79.3803231],
       [  43.653226 ,  -79.3831843],
       [  43.6689775,  -79.2902133],
       [  51.253775 ,  -85.323214 ],
       [  56.130366 , -106.346771 ],
 

In [114]:
weights = []
for f in Path("./models").iterdir():
    with f.open('r') as f:
        v1,v2,v3 = f.read().split("\n", maxsplit=2)
        v1 = float(v1)
        v2 = [float(x) for x in v2.strip("][").split(',')]
        v3 = v3.replace("\n", '').strip(" ").replace(" ",",")
        # print(v3)
        v3 = ast.literal_eval(v3)
        weights.append({"w":v1,"m":v2,"s": v3})
weights

[{'w': 0.12871855944038307,
  'm': [43.76732144514967, -79.45472948868485],
  's': [[0.00064723, 0.00224939], [0.00224939, 0.01114161]]},
 {'w': 0.17388597327363656,
  'm': [43.70529930597177, -79.49724923503747],
  's': [[0.00149265, 0.00037961], [0.00037961, 0.00278214]]},
 {'w': 0.19468116228540486,
  'm': [43.716764753044565, -79.27642642511333],
  's': [[0.00134813, 0.00168437], [0.00168437, 0.00292432]]},
 {'w': 0.10139717310898705,
  'm': [43.77751635130634, -79.26537783596987],
  's': [[0.00058834, 0.0004414], [0.0004414, 0.00322054]]},
 {'w': 0.1364843910303939,
  'm': [43.6634446067023, -79.44688986029712],
  's': [[0.00159994, 0.00187149], [0.00187149, 0.00305711]]},
 {'w': 0.26483274086119457,
  'm': [43.655643346822295, -79.38622062572544],
  's': [[0.000124694869, 9.48585435e-05], [9.48585435e-05, 0.000421826714]]}]

In [115]:
def mvg(w,m,s,x):
    a = 1.0/(np.sqrt(((2*np.pi)**2)*np.linalg.det(s)))
    b=x-m
    c=np.dot(np.dot(b.T, np.linalg.inv(s)),b)
    return w*a*np.exp(-0.5*c)

In [116]:
num = np.shape(s_loc)[0]
g=np.zeros(num)
d=np.zeros((num,num))

In [117]:
def dist(Lat1,Long1,Lat2,Long2):
    earth_radius = 3959*5280.0

    Lat1 = (Lat1)*(np.math.pi/180.0)
    Long1 = (Long1)*(np.math.pi/180.0)
    Lat2 = (Lat2)*(np.math.pi/180.0)
    Long2 = (Long2)*(np.math.pi/180.0)

    return 2.0*earth_radius*np.arcsin(np.sqrt(np.power(np.math.sin((Lat2-Lat1)/2.0),2.0)+np.math.cos(Lat1)*np.math.cos(Lat2)*np.power(np.math.sin((Long2 - Long1)/2.0),2.0)))

def checkPath(path_list, dist_list):
    l = np.argsort(dist_list)[0]
    if len(dist_list)==1 and dist_list[0] in path_list:
        return path_list
    else:
        if l in dist_list:
            return checkPath(path_list, np.argsort(dist_list)[1:])
        else:
            path_list.append(l)
            return path_list

In [118]:
idx = []
done = []
count = 0

for i, locs in enumerate(s_loc):
    s = 0
    for c in weights:
        globals()[f"g{i}"]=mvg(x=locs, **c)
        s+= globals()[f"g{i}"]
    g[i] = s
    for j in range(0, np.shape(s_loc)[0]):
        d[i,j] = dist(locs[0], locs[1], s_loc[j,0],s_loc[j,1])
        if d[i,j] ==0:
            d[i,j]=np.nan


In [119]:
d_sort = np.argsort(d, axis=1)
g3 = np.zeros((num, 3))
for j in range(0, num):
    g3[j,:]=g[d_sort[j,0:3]]

In [120]:
path = []
for M in g3:
    path = checkPath(path, M)
for m in path:
    for n in range(0, num):
        if g3[m][m] == g[n]:
            idx.append(n)
            break

In [121]:
for i in idx:
    with Path("./res.txt").open('w') as f:
        f.write(f"{str(s_loc[i,0])},{str(s_loc[i,1])}\n")
        f.write(f"{str(lat2)},{str(long2)}\n")

In [122]:
!cat res.txt

43.6603242,-79.3937162
43.65434450000001,-79.37972890000002


---

In [106]:
df = pd.read_csv('data/assault_lat_long.csv')
loc = list(df.to_records(index=False))

heatmap = gmaps.heatmap_layer(loc)
directions = gmaps.directions_layer((str(s_loc[i,0]), str(s_loc[i,1])), (str(lat2), str(long2)), travel_mode='WALKING')

In [123]:
figs.add_layer(gmaps.directions_layer((lat1,long1), (lat2, long2), travel_mode='WALKING'))

In [107]:
figs.add_layer(gmaps.heatmap_layer(loc))
figs.add_layer(gmaps.directions_layer((lat1,long1), (lat2, long2), travel_mode='WALKING'))

In [124]:
figs

Figure(layout=FigureLayout(height='420px'))

---

In [ ]:
kept = ['offence', 'Neighbourhood', 'Long', 'Lat']
data = pd.read_csv("data/Assault.csv")
data.head()

In [ ]:
data = data[data['occurrenceyear']>=2017]
# data = data[data.offence.str.contains('Assault')]
data = data[~data.offence.str.contains("Weapon")]

In [ ]:
finals = data.loc[:, kept]
finals

In [ ]:
finals.offence.unique()

In [ ]:
fin = finals.loc[finals['offence'].isin(['Assault', 'Aggravated Assault','Assault Bodily Harm','Assault - Resist/ Prevent Seiz','Assault - Force/Thrt/Impede','Crim Negligence Bodily Harm','Unlawfully Causing Bodily Harm'])]
fin.offence.unique()

In [ ]:
fin.info()

In [ ]:
plt.scatter(fin['Lat'], fin['Long'])
plt.colorbar()
plt.show()

In [ ]:
latlong = fin[['Lat', 'Long']]

In [ ]:
latlong.to_csv("data/assault_lat_long.csv", index=False, header=False)

---

In [ ]:
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
from pyspark.mllib.clustering import GaussianMixture

In [ ]:
data = sc.textFile("./data/assault_lat_long.csv")

In [ ]:
parsed = data.map(lambda line: np.array([float(x) for x in line.strip().split(',')]))
l=6

In [ ]:
gmm = GaussianMixture.train(parsed, l)

In [ ]:
for j in range(0,l):
    with open(f"./models/gmm.txt", 'a+') as f:
        f.write(str(gmm.weights[j])+"\n")
        f.write(str(gmm.gaussians[j].mu) + "\n")
        f.write(str(gmm.gaussians[j].sigma.toArray())+'\n')

In [ ]:
!cat ./models/gmm_4.txt

In [ ]:
!cat ./models/gmm.txt

In [ ]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X = sc_X.fit_transform(df.values)

In [ ]:
# Using the elbow method to find the optimal number of clusters
from sklearn.cluster import KMeans
wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 10), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Fitting K-Means to the dataset
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(X)

joblib.dump(kmeans, "models/kmeans.pkl")

In [ ]:
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(n_components = 2, kernel = 'rbf')
X = kpca.fit_transform(X)

In [ ]:
plt.scatter(X[y_kmeans == 0, 0], X[y_kmeans == 0, 1], s = 100, c = 'red')
plt.scatter(X[y_kmeans == 1, 0], X[y_kmeans == 1, 1], s = 100, c = 'blue')
plt.scatter(X[y_kmeans == 2, 0], X[y_kmeans == 2, 1], s = 100, c = 'green')
plt.scatter(X[y_kmeans == 3, 0], X[y_kmeans == 3, 1], s = 100, c = 'cyan')
plt.scatter(X[y_kmeans == 4, 0], X[y_kmeans == 4, 1], s = 100, c = 'magenta' )
plt.scatter(X[y_kmeans == 5, 0], X[y_kmeans == 5, 1], s = 100, c = 'black' )



plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 300, c = 'yellow', label = 'Centroids')
plt.title('Clusters of areas')
plt.xlabel('factor1')
plt.ylabel('factor2')
plt.legend()
plt.show()